In [1]:
%load_ext dotenv
%dotenv ../../src/.env

In [2]:
import sys
sys.path.append("../../src")

In [3]:
from utils.logger import get_logger
_logs = get_logger(__name__)
_logs.info("Notebook started successfully")

2025-11-13 22:23:20,062, 2927690305.py, 3, INFO, Notebook started successfully


In [4]:
import pandas as pd
import os
from glob import glob

In [5]:
import random

stock_files = glob(os.path.join(os.getenv('SRC_DIR'), "raw/*.csv"))


In [6]:
stock_files

['../../data/raw\\fundamentals.csv',
 '../../data/raw\\prices-split-adjusted.csv',
 '../../data/raw\\prices.csv',
 '../../data/raw\\securities.csv']

In [7]:
fundamentals = pd.read_csv(stock_files[0])
adj_prices = pd.read_csv(stock_files[1])
prices = pd.read_csv(stock_files[2])
securities = pd.read_csv(stock_files[3])

In [8]:
fundamentals.columns


Index(['Unnamed: 0', 'Ticker Symbol', 'Period Ending', 'Accounts Payable',
       'Accounts Receivable', 'Add'l income/expense items', 'After Tax ROE',
       'Capital Expenditures', 'Capital Surplus', 'Cash Ratio',
       'Cash and Cash Equivalents', 'Changes in Inventories', 'Common Stocks',
       'Cost of Revenue', 'Current Ratio', 'Deferred Asset Charges',
       'Deferred Liability Charges', 'Depreciation',
       'Earnings Before Interest and Tax', 'Earnings Before Tax',
       'Effect of Exchange Rate',
       'Equity Earnings/Loss Unconsolidated Subsidiary', 'Fixed Assets',
       'Goodwill', 'Gross Margin', 'Gross Profit', 'Income Tax',
       'Intangible Assets', 'Interest Expense', 'Inventory', 'Investments',
       'Liabilities', 'Long-Term Debt', 'Long-Term Investments',
       'Minority Interest', 'Misc. Stocks', 'Net Borrowings', 'Net Cash Flow',
       'Net Cash Flow-Operating', 'Net Cash Flows-Financing',
       'Net Cash Flows-Investing', 'Net Income', 'Net Income Ad

In [9]:
prices.columns, adj_prices.columns

(Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object'),
 Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object'))

In [10]:
securities.columns

Index(['Ticker symbol', 'Security', 'SEC filings', 'GICS Sector',
       'GICS Sub Industry', 'Address of Headquarters', 'Date first added',
       'CIK'],
      dtype='object')

In [11]:
# Standardize column names
for df in [fundamentals, prices, adj_prices, securities]:
    df.columns = df.columns.str.lower().str.strip()

In [12]:
fundamentals.columns

Index(['unnamed: 0', 'ticker symbol', 'period ending', 'accounts payable',
       'accounts receivable', 'add'l income/expense items', 'after tax roe',
       'capital expenditures', 'capital surplus', 'cash ratio',
       'cash and cash equivalents', 'changes in inventories', 'common stocks',
       'cost of revenue', 'current ratio', 'deferred asset charges',
       'deferred liability charges', 'depreciation',
       'earnings before interest and tax', 'earnings before tax',
       'effect of exchange rate',
       'equity earnings/loss unconsolidated subsidiary', 'fixed assets',
       'goodwill', 'gross margin', 'gross profit', 'income tax',
       'intangible assets', 'interest expense', 'inventory', 'investments',
       'liabilities', 'long-term debt', 'long-term investments',
       'minority interest', 'misc. stocks', 'net borrowings', 'net cash flow',
       'net cash flow-operating', 'net cash flows-financing',
       'net cash flows-investing', 'net income', 'net income ad

In [13]:
# Rename key columns for consistency
fundamentals = fundamentals.rename(columns={
    'ticker symbol': 'ticker',
    'period ending': 'date'
})
prices = prices.rename(columns={'symbol': 'ticker'})
adj_prices = adj_prices.rename(columns={'symbol': 'ticker'})
securities = securities.rename(columns={'ticker symbol': 'ticker'})

In [14]:
securities.columns

Index(['ticker', 'security', 'sec filings', 'gics sector', 'gics sub industry',
       'address of headquarters', 'date first added', 'cik'],
      dtype='object')

In [15]:
adj_prices.shape

(851264, 7)

In [16]:
fundamentals.shape

(1781, 79)

In [17]:
securities.shape

(505, 8)

In [18]:
# Unique tickers in each dataset
tickers_fund = set(fundamentals['ticker'].unique())
tickers_adj_price = set(adj_prices['ticker'].unique())
tickers_price = set(adj_prices['ticker'].unique())
tickers_sec = set(securities['ticker'].unique())

In [19]:
# --- Compare fundamentals vs adjusted prices ---
print("Tickers in fundamentals but not in prices:", tickers_fund - tickers_price)
print("Tickers in prices but not in fundamentals:", tickers_price - tickers_fund)

Tickers in fundamentals but not in prices: {'UA'}
Tickers in prices but not in fundamentals: {'AET', 'ICE', 'PSA', 'RIG', 'RAI', 'FOXA', 'ESRX', 'A', 'ENDP', 'JCI', 'L', 'AES', 'TROW', 'WBA', 'FOX', 'KHC', 'PRGO', 'ADP', 'BEN', 'DTE', 'ACN', 'MNK', 'FTV', 'CBS', 'DOW', 'NI', 'RF', 'FTI', 'IR', 'COH', 'URI', 'CMCSA', 'AGN', 'GS', 'MDT', 'JNJ', 'ORCL', 'BLK', 'GE', 'RTN', 'NOC', 'LNC', 'GOOG', 'MSI', 'EVHC', 'GOOGL', 'TWX', 'COP', 'FITB', 'CA', 'NRG', 'SLB', 'PLD', 'PXD'}


In [20]:
# --- Compare fundamentals vs securities ---
print("Tickers in fundamentals but not in securities:", tickers_fund - tickers_sec)
print("Tickers in securities but not in fundamentals:", len(tickers_sec - tickers_fund))


Tickers in fundamentals but not in securities: set()
Tickers in securities but not in fundamentals: 57


In [21]:
# --- Compare prices vs securities ---
print("Tickers in prices but not in securities:", tickers_price - tickers_sec)
print("Tickers in securities but not in prices:", tickers_sec - tickers_price)

Tickers in prices but not in securities: set()
Tickers in securities but not in prices: {'UA', 'MS', 'BF.B', 'BRK.B'}


In [22]:
# --- Compare prices vs adj_prices ---
print("Tickers in prices but not in adjusted prices:", tickers_price - tickers_adj_price)
print("Tickers in adjusted prices but not in prices:", tickers_adj_price - tickers_price)

Tickers in prices but not in adjusted prices: set()
Tickers in adjusted prices but not in prices: set()


In [23]:
common_tickers = tickers_fund & tickers_adj_price & tickers_sec
print("Number of tickers present in all three datasets:", len(common_tickers))

Number of tickers present in all three datasets: 447


In [24]:
# selecting tickers that are present in all the datasets
fundamentals = fundamentals[fundamentals['ticker'].isin(common_tickers)]
adj_prices = adj_prices[adj_prices['ticker'].isin(common_tickers)]
securities = securities[securities['ticker'].isin(common_tickers)]

In [25]:
# convert the date to datetime column
fundamentals['date'] = pd.to_datetime(fundamentals['date'])
adj_prices['date']   = pd.to_datetime(adj_prices['date'])


In [26]:

# Filter fundamentals *up to* the cutoff date
cutoff_date = pd.to_datetime("2016-01-01")
fundamentals_cut = fundamentals[fundamentals['date'] <= cutoff_date].copy()


In [27]:
fundamentals_cut.shape

(1556, 79)

In [28]:
# For each ticker, keep the latest available record
fundamentals_latest = (
    fundamentals_cut
    .sort_values(['ticker', 'date'])
    .groupby('ticker')
    .tail(1)          # last record per ticker
    .reset_index(drop=True)
)

In [29]:
fundamentals_latest.shape

(447, 79)

In [30]:
fund_sec = pd.merge(fundamentals_latest, securities, on='ticker', how='left')

In [31]:
fund_sec.columns

Index(['unnamed: 0', 'ticker', 'date', 'accounts payable',
       'accounts receivable', 'add'l income/expense items', 'after tax roe',
       'capital expenditures', 'capital surplus', 'cash ratio',
       'cash and cash equivalents', 'changes in inventories', 'common stocks',
       'cost of revenue', 'current ratio', 'deferred asset charges',
       'deferred liability charges', 'depreciation',
       'earnings before interest and tax', 'earnings before tax',
       'effect of exchange rate',
       'equity earnings/loss unconsolidated subsidiary', 'fixed assets',
       'goodwill', 'gross margin', 'gross profit', 'income tax',
       'intangible assets', 'interest expense', 'inventory', 'investments',
       'liabilities', 'long-term debt', 'long-term investments',
       'minority interest', 'misc. stocks', 'net borrowings', 'net cash flow',
       'net cash flow-operating', 'net cash flows-financing',
       'net cash flows-investing', 'net income', 'net income adjustments',
    

In [32]:
keep_cols = [
    # Identity
    'ticker', 'gics sector',

    # Financial Ratios
    'current ratio', 'quick ratio', 'cash ratio', 
    'gross margin', 'operating margin', 'profit margin',
    'pre-tax margin', 'pre-tax roe', 'after tax roe',
    'earnings per share',

    # Balance Sheet (high-level)
    'total assets', 'total current assets', 'total current liabilities',
    'total liabilities', 'total equity', 'long-term debt',
    'inventory', 'investments', 'cash and cash equivalents',
    'fixed assets', 'intangible assets', 'goodwill',

    # Cash Flow
    'net cash flow-operating', 'net cash flows-investing',
    'net cash flows-financing', 'capital expenditures',

    # Income Statement
    'total revenue', 'gross profit', 'operating income',
    'earnings before tax', 'net income'
]


In [33]:
fund_sec.columns = fund_sec.columns.str.lower().str.strip()

In [34]:
# Filter only the columns that exist (avoids KeyErrors)
keep_cols_final = [col for col in keep_cols if col in fund_sec.columns]
print("Keeping", len(keep_cols_final), "columns out of", len(fund_sec.columns))

Keeping 33 columns out of 86


In [35]:
fund_sec_clean = fund_sec[keep_cols_final].copy()

In [36]:
print(f"Original shape: {fund_sec.shape}")
print(f"Cleaned shape:  {fund_sec_clean.shape}")
fund_sec_clean.head()

Original shape: (447, 86)
Cleaned shape:  (447, 33)


,ticker,gics sector,current ratio,quick ratio,cash ratio,gross margin,operating margin,profit margin,pre-tax margin,pre-tax roe,...,goodwill,net cash flow-operating,net cash flows-investing,net cash flows-financing,capital expenditures,total revenue,gross profit,operating income,earnings before tax,net income
0,AAL,Industrials,73.0,67.0,51.0,73.0,15.0,19.0,11.0,82.0,...,4.091000e+09,6.249000e+09,-5.594000e+09,-1.259000e+09,-6.151000e+09,4.099000e+10,2.989400e+10,6.204000e+09,4.616000e+09,7.610000e+09
1,AAP,Consumer Discretionary,130.0,22.0,3.0,45.0,9.0,5.0,8.0,39.0,...,9.954260e+08,7.089910e+08,-2.288237e+09,5.759110e+08,-2.284460e+08,9.843861e+09,4.453613e+09,8.517100e+08,7.813940e+08,4.938250e+08
2,AAPL,Information Technology,111.0,108.0,52.0,40.0,30.0,23.0,31.0,61.0,...,5.116000e+09,8.126600e+10,-5.627400e+10,-1.771600e+10,-1.124700e+10,2.337150e+11,9.362600e+10,7.123000e+10,7.251500e+10,5.339400e+10
3,ABBV,Health Care,150.0,134.0,77.0,80.0,33.0,23.0,29.0,168.0,...,1.316800e+10,7.535000e+09,-1.293600e+10,5.752000e+09,-5.320000e+08,2.285900e+10,1.835900e+10,7.537000e+09,6.645000e+09,5.144000e+09
4,ABC,Health Care,94.0,49.0,10.0,3.0,0.0,0.0,0.0,44.0,...,4.144391e+09,3.922228e+09,-2.931165e+09,-6.321340e+08,-2.315850e+08,1.359618e+11,3.529313e+09,4.222200e+08,2.689640e+08,-1.381650e+08


In [37]:
# Change the name of price data as it is processed
time_series_dataset = adj_prices

In [38]:
time_series_dataset = time_series_dataset.sort_values(['ticker', 'date']).reset_index(drop=True)

In [39]:
time_series_dataset.shape

(760913, 7)

In [40]:
# Daily returns %
time_series_dataset['return_pct'] = time_series_dataset.groupby('ticker')['close'].pct_change() * 100


In [41]:
# Rolling volatility
time_series_dataset['vol_5d'] = time_series_dataset.groupby('ticker')['return_pct'].rolling(5).std().reset_index(0, drop=True)
time_series_dataset['vol_10d'] = time_series_dataset.groupby('ticker')['return_pct'].rolling(10).std().reset_index(0, drop=True)

In [42]:
# Momentum
time_series_dataset['momentum_5d'] = time_series_dataset.groupby('ticker')['close'].diff(5)
time_series_dataset['momentum_10d'] = time_series_dataset.groupby('ticker')['close'].diff(10)

In [43]:
# Moving averages
time_series_dataset['sma_5d'] = time_series_dataset.groupby('ticker')['close'].rolling(5).mean().reset_index(0, drop=True)
time_series_dataset['sma_10d'] = time_series_dataset.groupby('ticker')['close'].rolling(10).mean().reset_index(0, drop=True)


In [44]:
# High-low range %
time_series_dataset['hl_range_pct'] = (time_series_dataset['high'] - time_series_dataset['low']) / time_series_dataset['close'] * 100

In [45]:
# Open-close change %
time_series_dataset['oc_change_pct'] = (time_series_dataset['close'] - time_series_dataset['open']) / time_series_dataset['open'] * 100

In [46]:
# Rolling average volume
time_series_dataset['avg_volume_5d'] = time_series_dataset.groupby('ticker')['volume'].rolling(5).mean().reset_index(0, drop=True)
time_series_dataset['avg_volume_10d'] = time_series_dataset.groupby('ticker')['volume'].rolling(10).mean().reset_index(0, drop=True)

In [47]:
# Fill NaNs caused by rolling windows
time_series_dataset.fillna(0, inplace=True)

In [48]:
time_series_dataset.columns

Index(['date', 'ticker', 'open', 'close', 'low', 'high', 'volume',
       'return_pct', 'vol_5d', 'vol_10d', 'momentum_5d', 'momentum_10d',
       'sma_5d', 'sma_10d', 'hl_range_pct', 'oc_change_pct', 'avg_volume_5d',
       'avg_volume_10d'],
      dtype='object')

In [49]:
# Filter up to cut-off date to avoid leakage
ts_filtered = time_series_dataset[time_series_dataset['date'] <= cutoff_date]

In [50]:
# Aggregate per ticker
agg_features = ts_filtered.groupby('ticker').agg(
    # Price stats
    last_close=('close', 'last'),
    max_close=('close', 'max'),
    min_close=('close', 'min'),

    # Returns
    avg_return=('return_pct', 'mean'),
    std_return=('return_pct', 'std'),

    # Volatility
    avg_vol_5d=('vol_5d', 'mean'),
    avg_vol_10d=('vol_10d', 'mean'),

    # Momentum
    last_momentum_5d=('momentum_5d', 'last'),
    last_momentum_10d=('momentum_10d', 'last'),

    # Moving averages
    last_sma_5d=('sma_5d', 'last'),
    last_sma_10d=('sma_10d', 'last'),

    # Price range / price change
    avg_hl_range_pct=('hl_range_pct', 'mean'),
    avg_oc_change_pct=('oc_change_pct', 'mean'),

    # Volume
    last_avg_volume_5d=('avg_volume_5d', 'last'),
    last_avg_volume_10d=('avg_volume_10d', 'last')
).reset_index()

In [51]:
# Merge aggregated features into ticker-level dataset
ticker_features = pd.merge(fund_sec_clean, agg_features, on='ticker', how='left')

In [52]:
print(" Ticker-level dataset enriched with aggregated time-series features.")
print("Shape:", ticker_features.shape)

 Ticker-level dataset enriched with aggregated time-series features.
Shape: (447, 48)


In [53]:
missing_summary = ticker_features.isnull().sum()
print(missing_summary[missing_summary > 0])

current ratio          75
quick ratio            75
cash ratio             75
earnings per share     34
last_close              1
max_close               1
min_close               1
avg_return              1
std_return              1
avg_vol_5d              1
avg_vol_10d             1
last_momentum_5d        1
last_momentum_10d       1
last_sma_5d             1
last_sma_10d            1
avg_hl_range_pct        1
avg_oc_change_pct       1
last_avg_volume_5d      1
last_avg_volume_10d     1
dtype: int64


In [54]:
numeric_cols = ticker_features.select_dtypes(include='number').columns

In [55]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=5)
ticker_features[numeric_cols] = imputer.fit_transform(ticker_features[numeric_cols])


In [56]:
# Identify categorical columns, **excluding 'ticker'**
categorical_cols = ticker_features.select_dtypes(include=['object', 'category']).columns.tolist()
categorical_cols = [col for col in categorical_cols if col != 'ticker']

In [57]:
categorical_cols

['gics sector']

In [58]:
# Impute missing values with most frequent category
for col in categorical_cols:
    ticker_features[col] = ticker_features[col].fillna(ticker_features[col].mode().iloc[0])

In [59]:
PROCESSED_DIR = os.path.join(os.getenv('SRC_DIR'), "processed")
os.makedirs(PROCESSED_DIR, exist_ok=True)

In [60]:
time_series_path = os.path.join(PROCESSED_DIR, "time_series_dataset.csv")
ticker_features_path = os.path.join(PROCESSED_DIR, "ticker_features.csv")

In [61]:
time_series_dataset.to_csv(time_series_path, index=False)
ticker_features.to_csv(ticker_features_path, index=False)